 # Big-Data Analytics
 # Group 25
 

The goal of the project is to process a raw dataset from beginning to end, going through each of the steps in the data science pipeline. The ongoing Coronavirus crisis has upended not only our semester, but our lives, so our group could not think of a more fitting topic to examine with this project. Amidst the rapidly evolving threat, the most up-to-date sources have often been social media platforms online both for front-line ER doctors sharing clinical procedures (https://www.nytimes.com/2020/03/18/well/live/coronavirus-doctors-facebook-twitter-social-media-covid.html) and for the broader scientific community and reporters, sharing insights and ideas to combat the crisis. 

Its timeliness and reputation, as well as the company's relative openness to sharing data with developers via their API, made Twitter a logical source of data for this project.

Early in the crisis, many people (though populists most prominently, with their anti-expertise bias) the world over claimed "the media" was overreacting to the crisis, and sensationalizing the threat. While this interpretation has been clearly born out as false, our group is interested in how closely the "buzz" around the virus correlates with the actual rates of infection. Much has been made about the virus's exponential spread: do posts about the virus in affected regions follow the same pattern? How can we understand not only the patterns within countries but the differences between countries?

The deliverable of the project consists of a Jupyter notebook that includes the
code used for exploring, processing, analyzing, and visualizing the data set and
the data set itself (this can be download link). If the data set is very large, a
sample of the data set can also be appropriate. 

# Data collection/acquisition

In [ ]:
#pulling the 30 day data (with & without geolocation data) from Twitter


# Data preprocessing/cleaning




In [ ]:
#counts by region
#plotting day-by-day bar charts counting mentions?
#plotting on map of Italy with dot size showing the number of mentions


# Exploratory Data Analysis

# Model/algorithm building

In [ ]:
# @Luthy & @Joel: any good ideas for modeling this?



 # Data visualization & interpretation

In [ ]:
#see how the rates correlate when we control for twitter DAU  & MAU (daily/monthly active users)